In [2]:
import numpy
import pandas as pd
import xpress as xp


## Data Exploration

In [4]:
# read-data
TransactionsInput = pd.read_excel("231013_Transactions_Input.xlsx")

In [5]:
FuardCases = pd.read_excel("231013_Fraud_Cases.xlsx")
CustomerBase = pd.read_excel("231013_Customer_Base.xlsx")

In [6]:
TransactionsInput["description"].nunique()

347

In [7]:
TransactionsInput

,transaction_id,description,Amount,category,date,month,customer_id,type,In_or_Out,bank_to,bank_from,transac_prob,description_prob,priority
0,1,Salary Deposit - XYZ Corporation,11474.33021,Income,2023-10-01,October,17699,income,paid_in,bank_A,bank_A,NaN,NaN,3
1,2,Online Retailer - Electronics Purchase,199.99000,Online Shopping,2023-10-01,October,10298,spending,paid_out,bank_A,bank_E,0.78,0.25,3
2,3,Utilities Payment - Internet Service,60.00000,Utilities,2023-10-01,October,10298,spending,paid_out,bank_C,bank_E,0.30,0.22,1
3,4,Rent Payment,1500.00000,Housing,2023-10-01,October,14507,spending,paid_out,bank_D,bank_E,0.57,0.53,3
4,5,Paycheck Deposit - Company Name,10480.68449,Income,2023-10-01,October,14507,income,paid_in,bank_E,bank_D,NaN,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315177,315178,Grocery Shopping - Super Value Mart,55.25000,Groceries,2024-07-31,July,15026,spending,paid_out,bank_C,bank_B,0.42,0.17,1
315178,315179,JCPenney - Discount Fashion,40.00000,Shopping,2024-07-31,July,11908,spending,paid_out,bank_D,bank_A,0.36,0.36,1
315179,315180,Gas Station - Fuel Purchase,40.45000,Transportation,2024-07-31,July,10291,spending,paid_out,bank_D,bank_D,0.28,0.64,1
315180,315181,ATM Withdrawal - PQR Bank ATM,80.00000,Cash Withdrawal,2024-07-31,July,18543,spending,paid_out,NaN,bank_A,0.24,0.23,2


In [8]:
TransactionsInput_clean = TransactionsInput.drop('description_prob', axis=1)

In [ ]:
# TransactionsInput_clean = TransactionsInput_clean[["In_or_Out"] == "paid_in"]

## Create a problem with xpress module

#### The basic model in a single day. (Ignore the learn model)

In [19]:
# just use for testing
customer_num = 10

In [27]:
# data (parameeters) needed in this problem
# NOTE THAT: very strightforward way in this case as this is the idea stage.
# still need to abstract the single instead of a whole data set
x_length = len(TransactionsInput_clean["transaction_id"])
y_length = len(TransactionsInput_clean["transaction_id"])
Prob_transac = TransactionsInput_clean["transac_prob"]
# Prob_description = TransactionsInput_clean["description_prob"]
Prob_customer = CustomerBase["customer_prob"]


Bank = ["A", "B", "C", "D", "E"]
Investigarion_team = [8, 12, 10, 10, 10]

Priority = [1,2,3,4]
Investigation_time = [0.25, 0.5, 1, 2]
Investigation_hire_exter = [40, 60, 100, 150]
Investigation_success = [0.5, 0.6, 0.7, 0.8]


In [13]:
# Create a problem called transaction

prob = xp.problem(name='transaction') 

Using the license file found in your Xpress installation. If you want to use this license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('D:/APP/Xpress/bin/xpauth.xpr')


In [34]:
# Create the x variables
x = xp.var(vartype=xp.binary, name="x") # if we want to investigate the transaction i
f = xp.var(vartype=xp.binary, name="f") # if the in vestigation label transaction i is fraud
y = xp.var(vartype=xp.binary, name="y") #

# Add the variables to the problem

prob.addVariable(x,f,y)

?1030 Error: Duplicate column names are not allowed - column 1 would have same name as column 2


SolverError: ?1030 Error: Duplicate column names are not allowed - column 1 would have same name as column 2

In [15]:
# add objective value
# expect value on investigation cost
prob.addObjective(i for i in x_length x[i]*)

In [16]:
# add constraints
prob.addConstraint()